# Inserting Training and Test Data into a Db2 Table

In [ ]:
#Establishing Connection to Database

import pandas as pd
import ibm_db_dbi
import ibm_db
from project_lib import Project 

# Define connection string and connect to database
project = Project.access()
LocalDB2_credentials = project.get_connection(name = "CSSDB3")

bluedb_connection = ibm_db.connect("DATABASE={};HOSTNAME={};PORT={};PROTOCOL=TCPIP;UID={};PWD={}".format(LocalDB2_credentials['database'],
                                                                                                         LocalDB2_credentials['host'],
                                                                                                         LocalDB2_credentials['port'],
                                                                                                         LocalDB2_credentials['username'],
                                                                                                         LocalDB2_credentials['password']),"","")
dbi_bluedb_connection = ibm_db_dbi.Connection(bluedb_connection)

## Insert Training Data

In [ ]:
#Reading in CSV file with raw data and preparing to write to DB2 

train_data = pd.read_csv('/project_data/data_asset/customer_full_summary_latest.csv')

train_data = test_data.where(pd.notnull(train_data),None)

tuple_of_tuples = tuple([tuple(x) for x in train_data.values])

In [ ]:
#Creating string concatenations for sql query to write data 

type_mapping = {'int64': 'DOUBLE',
                'float64': 'DOUBLE',
               'object': 'VARCHAR(90)',
                'datetime64[ns]': 'VARCHAR(90)'
               }

value_string = ', '.join(['?' for i in range(len(train_data.columns))])
dtype_mapping_4table = [type_mapping[str(x)] for x in list(train_data.dtypes)]
table_creation_column_string =  ', '.join('{0} {1} '.format(str(x),str(y)) for x,y in zip(train_data.columns,dtype_mapping_4table))

In [ ]:
sql = 'CREATE TABLE DSE.CUST_SEG_DATA_TRAIN({});'.format(table_creation_column_string)

stmt = ibm_db.prepare(bluedb_connection, sql)

ibm_db.execute(stmt)

In [ ]:
value_string = ', '.join(['?' for i in range(len(train_data.columns))])

sql = "INSERT INTO DSE.CUST_SEG_DATA_TRAIN VALUES({})".format(value_string)

stmt = ibm_db.prepare(bluedb_connection, sql)

ibm_db.execute_many(stmt, tuple_of_tuples)

In [ ]:
# Validate successful data import
input_df = pd.read_sql('select * from DSE.CUST_SEG_DATA_TRAIN',con = dbi_bluedb_connection)

In [ ]:
input_df.shape

In [ ]:
input_df.head()

## Insert Test Data

In [ ]:
#Reading in CSV file with raw data and preparing to write to DB2 

test_data = pd.read_csv('/project_data/data_asset/test_data_10K.csv')

test_data = test_data.where(pd.notnull(test_data),None)

tuple_of_tuples = tuple([tuple(x) for x in test_data.values])

In [ ]:
#Creating string concatenations for sql query to write data 

type_mapping = {'int64': 'DOUBLE',
                'float64': 'DOUBLE',
               'object': 'VARCHAR(90)',
                'datetime64[ns]': 'VARCHAR(90)'
               }

value_string = ', '.join(['?' for i in range(len(test_data.columns))])
dtype_mapping_4table = [type_mapping[str(x)] for x in list(test_data.dtypes)]
table_creation_column_string =  ', '.join('{0} {1} '.format(str(x),str(y)) for x,y in zip(test_data.columns,dtype_mapping_4table))

In [ ]:
sql = 'CREATE TABLE DSE.CUST_SEG_DATA_TEST({});'.format(table_creation_column_string)

stmt = ibm_db.prepare(bluedb_connection, sql)

ibm_db.execute(stmt)

In [ ]:
value_string = ', '.join(['?' for i in range(len(test_data.columns))])

sql = "INSERT INTO DSE.CUST_SEG_DATA_TEST VALUES({})".format(value_string)

stmt = ibm_db.prepare(bluedb_connection, sql)

ibm_db.execute_many(stmt, tuple_of_tuples)

In [ ]:
# Validate successful data import
input_df = pd.read_sql('select * from DSE.CUST_SEG_DATA_TEST',con = dbi_bluedb_connection)

In [ ]:
input_df.shape

In [ ]:
input_df.head()